In [1]:
import os
import zlib
import pandas as pd
import numpy as np
import re
import lzo

ModuleNotFoundError: No module named 'lzo'

In [ ]:
import os

: 

In [2]:
Source = r'save_1'

In [15]:
class Seg:
    
    def __init__(self, index, record):
        self.index = index
        self.start = record.start()
        self.end = record.end()
        self.content = record.group()
        self.magic = self.content[:4]
        self.block_size = Save.bytes2int(self.content[4:8])
        self.compressed_size = Save.bytes2int(self.content[8:12])
        self.uncompressed_size = Save.bytes2int(self.content[12:16])
        self.compressed_size2 = Save.bytes2int(self.content[16:20])
        self.uncompressed_size2 = Save.bytes2int(self.content[20:24])
        self.compressed = self.content[24:]
        self.uncompressed = lzo.decompress(self.compressed, False, self.uncompressed_size)
        self.summary = {'index' : self.index,
                        'magic': self.magic,
                        'block_size': self.block_size,
                        'compressed_size': self.compressed_size,
                        'compressed_size2': self.compressed_size2,
                        'uncompressed_size': self.uncompressed_size,
                        'uncompressed_size2': self.uncompressed_size2,
                        'actual_compressed_length': len(self.compressed),
                        'actual_uncompressed_length': len(self.uncompressed)}

        
class Save:
    
    @staticmethod
    def int2bytes(i, length, byteorder='little'):
        return i.to_bytes(length, byteorder=byteorder)

    @staticmethod
    def bytes2int(b, byteorder='little'):
        return int.from_bytes(b, byteorder=byteorder)
    
    def __init__(self, filename):
        self.filename = filename
        with open(filename, 'rb') as s:
            self.contents = s.read()
        self.segs = self.find_segs()
        self.header = self.contents[:self.segs[0].start]
            
    def find_segs(self):
        search = re.finditer(b'\xc1\x83\\x2a\x9e.*?\x11\x00\x00(?=\xc1\x83\\x2a\x9e)', self.contents, flags=re.I|re.DOTALL)
        return [Seg(index, _) for (index, _) in enumerate(search)]

In [16]:
s = Save(Source)

<ipython-input-15-9f77eb086fab>:15: DeprecationWarning: PY_SSIZE_T_CLEAN will be required for '#' formats
  self.uncompressed = lzo.decompress(self.compressed, False, self.uncompressed_size)


In [29]:
s.segs[5].compressed

b'\r\x00m_TemplateName\x00a\x00\rP\x00p\x01\x05Property[\x01\x00\x00\x1b\x97\x00\x00\x00\x13\\\x00\x0fDarkEvent_WildHuntS\x02\x00\x00\x07T\x00\x02Weigh\xb9\x01\x0cX\x01\x01IntP*\x15\x01\x04L\x02i\x00\x05Q\x00\tL\x00\x05ObjectIDJ\x01\x00\x00:\xc0\x00\x02]\x02\x00\x00\x1aT\x04\x0ePreviousHistoryFrP\x16\x02IndexL\x03<\x04\x01`\x04\n\x05\x00\x00\x00None\x00\x00\x00\x00\x00\x11\x00\x00'